In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import DataFrame,functions as f
from env_vars import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY

spark = SparkSession.builder \
    .appName("S3 to Spark DataFrame") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .getOrCreate()

years = ["2000_", "2010_", "2020_"]
bucket_name = 'redistricting-data-2024'
object_name = 'cleaned_data.csv'

your 131072x1 screen size is bogus. expect trouble
24/08/26 10:36:07 WARN Utils: Your hostname, LAPTOP-E2047RF4 resolves to a loopback address: 127.0.1.1; using 192.168.157.37 instead (on interface eth0)
24/08/26 10:36:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 10:36:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df: DataFrame
for i in range(len(years)):
    file_name = f"{years[i]}" + object_name
    s3_path = f's3a://{bucket_name}/{file_name}'
    if i ==  0:
        df = spark.read.csv(s3_path, header=True, inferSchema=True)
    else:
        df = df.union(spark.read.csv(s3_path, header=True, inferSchema=True))

In [56]:
df.show()

+---------+------------+------+----------------+-----------+----------------------+---------------------------------------+-----------+--------------------------------------+-----------+-----------------+------------------+----------------------+----+
|State Abv|Summary Code|Region|Total Population|White Alone|African-American Alone|American Indian and Alaska Native Alone|Asian Alone|Native Hawaiian/Pacific Islander Alone|Other Alone|Two or More Races|Hispanic or Latino|Not Hispanic or Latino|year|
+---------+------------+------+----------------+-----------+----------------------+---------------------------------------+-----------+--------------------------------------+-----------+-----------------+------------------+----------------------+----+
|       AL|          40|     3|         4447100|    3162808|               1155930|                                  22430|      31346|                                  1409|      28998|            44179|           4371270|               433218

In [61]:
df1 = spark.createDataFrame(df.where(df["year"] == 2000).select("State Abv", "Total Population", "year")\
    .orderBy(f.desc(df["Total Population"])).take(5))
df2 = spark.createDataFrame(df.where(df["year"] == 2010).select("State Abv", "Total Population", "year")\
    .orderBy(f.desc(df["Total Population"])).take(5))
df3 = spark.createDataFrame(df.where(df["year"] == 2020).select("State Abv", "Total Population", "year")\
    .orderBy(f.desc(df["Total Population"])).take(5))

In [92]:
df_top5_state_population_by_year = df.join(df1, df["State Abv"] == df1["State Abv"])\
    .union(df.join(df2, df["State Abv"] == df2["State Abv"]))\
    .union(df.join(df3, df["State Abv"] == df3["State Abv"]))\
    .groupBy(df["State Abv"], df["Total Population"], df["year"])\
    .count()\
    .select("State Abv", "Total Population", "year").orderBy("year")

+---------+----------------+----+
|State Abv|Total Population|year|
+---------+----------------+----+
|       CA|        33871648|2000|
|       TX|        20851820|2000|
|       NY|        18976457|2000|
|       IL|        12419293|2000|
|       FL|        15982378|2000|
|       PA|        12281054|2000|
|       CA|        37253956|2010|
|       TX|        25145561|2010|
|       NY|        19378102|2010|
|       IL|        12830632|2010|
|       FL|        18801310|2010|
|       PA|        12702379|2010|
|       CA|        39538223|2020|
|       TX|        29145505|2020|
|       NY|        20201249|2020|
|       FL|        21538187|2020|
|       IL|        12812508|2020|
|       PA|        13002700|2020|
+---------+----------------+----+



In [97]:
df_top_region_population= df.groupBy("Region", "Year").agg(f.sum("Total Population").alias("Total_Population")).orderBy("year", "Region")

+------+----+----------------+
|Region|Year|Total_Population|
+------+----+----------------+
|     1|2000|        53594378|
|     2|2000|        64392776|
|     3|2000|       100236820|
|     4|2000|        63197932|
|     9|2000|         3808610|
|     1|2010|        55317240|
|     2|2010|        66927001|
|     3|2010|       114555744|
|     4|2010|        71945553|
|     9|2010|         3725789|
|     1|2020|        57609148|
|     2|2020|        68985454|
|     3|2020|       126266107|
|     4|2020|        78588572|
|     9|2020|         3285874|
+------+----+----------------+

